In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

# Import XGBoost (after installing)
from xgboost import XGBRegressor

import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'xgboost'